In [1]:
!pip install duckdb

In [48]:
import duckdb
import time

In [49]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [24]:
from py_duckdb.similarity_join import tokenizers
from py_duckdb.similarity_join import jaccard_join
from py_duckdb.similarity_join import jaccard_join_brute_force

In [5]:
con = duckdb.connect(database=':memory:')

# Source data

In [6]:
con.execute("drop table if exists src1").execute(
    "CREATE TABLE src1 AS "
    "SELECT id as rid, concat(given_name, ' ', surname, ' ', date_of_birth) as val "
    "FROM 'data/S1_clean_.csv'"
).execute("select * from src1").fetchall()

[('S1_0', 'joshua morrison 19101123'),
 ('S1_1', 'jordan white 19371126'),
 ('S1_2', 'emmerson lock 19211129'),
 ('S1_3', 'alexandra grosser 19720305'),
 ('S1_4', 'michael wuchatsch 19190110'),
 ('S1_5', 'emmerson loyck 19211129'),
 ('S1_6', 'rhys schuetz 19440909'),
 ('S1_7', 'joshua greenj 19790110'),
 ('S1_8', 'olivia hobson 19760812'),
 ('S1_9', 'michael lierach 19360816'),
 ('S1_10', 'elisabett domiten 19081008'),
 ('S1_11', 'genoveffa hylander 19071008')]

In [7]:
con.execute("drop table if exists src2").execute(
    "CREATE TABLE src2 AS "
    "SELECT id as rid, concat(given_name, ' ', surname, ' ', date_of_birth) as val "
    "FROM 'data/S2_clean_.csv'"
).execute("select * from src2").fetchall()

[('S2_0', 'braecon schuetz 19440909'),
 ('S2_1', 'alexandra grosvenor 19930305'),
 ('S2_2', 'michael liersch 19360816'),
 ('S2_3', 'emmeron loyk 19321129'),
 ('S2_4', 'olivia hobson 19760812'),
 ('S2_5', 'joshua green 19010219'),
 ('S2_6', 'charlotte hyland 19340909'),
 ('S2_7', 'elisabet domitienn 19071008')]

In [8]:
con.execute("drop table if exists src3").execute(
    "CREATE TABLE src3 AS "
    "SELECT id as rid, concat(given_name, ' ', surname, ' ', date_of_birth) as val "
    "FROM 'data/S3_clean_.csv'"
).execute("select * from src3").fetchall()

[('S3_0', 'emmerson loyck 19211129'),
 ('S3_1', 'michel wuchatsch 19190110'),
 ('S3_3', 'liersch michael 19360816'),
 ('S3_4', 'charlotte hyland 19460401'),
 ('S3_5', 'braedon schuetz 19440909'),
 ('S3_6', 'olivia hobson 19760812'),
 ('S3_7', 'joshua green 19790110'),
 ('S3_8', 'keely clarke 19050410'),
 ('S3_9', 'joshua morriosn 19101123'),
 ('S3_11', 'genovefa hyllande 19071008')]

In [9]:
con.execute("drop view if exists srcall").execute(
    "create view srcall as "
    "select * from src1 "
    "union "
    "select * from src2 "
    "union "
    "select * from src3 "
).execute("select * from srcall").fetchall()

[('S3_0', 'emmerson loyck 19211129'),
 ('S3_1', 'michel wuchatsch 19190110'),
 ('S3_3', 'liersch michael 19360816'),
 ('S3_4', 'charlotte hyland 19460401'),
 ('S3_5', 'braedon schuetz 19440909'),
 ('S3_6', 'olivia hobson 19760812'),
 ('S3_7', 'joshua green 19790110'),
 ('S3_8', 'keely clarke 19050410'),
 ('S3_9', 'joshua morriosn 19101123'),
 ('S3_11', 'genovefa hyllande 19071008'),
 ('S2_0', 'braecon schuetz 19440909'),
 ('S2_1', 'alexandra grosvenor 19930305'),
 ('S2_2', 'michael liersch 19360816'),
 ('S2_3', 'emmeron loyk 19321129'),
 ('S2_4', 'olivia hobson 19760812'),
 ('S2_5', 'joshua green 19010219'),
 ('S2_6', 'charlotte hyland 19340909'),
 ('S2_7', 'elisabet domitienn 19071008'),
 ('S1_0', 'joshua morrison 19101123'),
 ('S1_1', 'jordan white 19371126'),
 ('S1_2', 'emmerson lock 19211129'),
 ('S1_3', 'alexandra grosser 19720305'),
 ('S1_4', 'michael wuchatsch 19190110'),
 ('S1_5', 'emmerson loyck 19211129'),
 ('S1_6', 'rhys schuetz 19440909'),
 ('S1_7', 'joshua greenj 19790110'

# Test: actors

In [27]:
# function args
l_table = 'srcall'
# r_table = 'srcall'
r_table = ''

l_key_attr = 'rid'
r_key_attr = 'rid'
l_join_attr = 'val'
r_join_attr = 'val'
# tokenizer = tokenizers.WordsTokzr()
tokenizer = tokenizers.QGramsTokzr(3)
threshold = 0.5
out_table_name = 'matches'
bf_out_table_name = 'bfmatches'

In [63]:
start_time = time.time()
jaccard_join(
    con, l_table, r_table, l_key_attr, r_key_attr, l_join_attr, r_join_attr, tokenizer, threshold, out_table_name
).execute(f"select * from {out_table_name}").fetchall()
end_time = time.time()
end_time - start_time

0.06242847442626953

In [64]:
start_time = time.time()
jaccard_join_brute_force(
    con, l_table, r_table, l_key_attr, r_key_attr, l_join_attr, r_join_attr, tokenizer, threshold, bf_out_table_name
).execute(f"select * from {bf_out_table_name}").fetchall()
end_time = time.time()
end_time - start_time

0.005068778991699219

In [65]:
con.execute(
    "select * "
    f"from {out_table_name} m "
    f"full outer join {bf_out_table_name} b on b.rid1 = m.rid1 and b.rid2 = m.rid2 "
    "where m.rid1 is null "
).fetchall()

[]

In [44]:
# function args
l_table = 'src1'
r_table = 'src2'

l_key_attr = 'rid'
r_key_attr = 'rid'
l_join_attr = 'val'
r_join_attr = 'val'
# tokenizer = tokenizers.WordsTokzr()
tokenizer = tokenizers.QGramsTokzr(3)
threshold = 0.5
out_table_name = 'matches'
bf_out_table_name = 'bfmatches'

In [45]:
jaccard_join(
    con, l_table, r_table, l_key_attr, r_key_attr, l_join_attr, r_join_attr, tokenizer, threshold, out_table_name
).execute(f"select * from {out_table_name}").fetchall()

[('S1_10', 'S2_7'),
 ('S1_3', 'S2_1'),
 ('S1_5', 'S2_3'),
 ('S1_6', 'S2_0'),
 ('S1_8', 'S2_4'),
 ('S1_9', 'S2_2')]

In [46]:
jaccard_join_brute_force(
    con, l_table, r_table, l_key_attr, r_key_attr, l_join_attr, r_join_attr, tokenizer, threshold, bf_out_table_name
).execute(f"select * from {bf_out_table_name}").fetchall()

[('S1_3', 'S2_1'),
 ('S1_5', 'S2_3'),
 ('S1_6', 'S2_0'),
 ('S1_8', 'S2_4'),
 ('S1_9', 'S2_2'),
 ('S1_10', 'S2_7')]

In [47]:
con.execute(
    "select * "
    f"from {out_table_name} m "
    f"full outer join {bf_out_table_name} b on b.rid1 = m.rid1 and b.rid2 = m.rid2 "
    "where m.rid1 is null "
).fetchall()

[]

# Test: NCVR

In [67]:
con.execute("drop table if exists src1").execute(
    "CREATE TABLE src1 AS "
    "SELECT id as rid, concat(entity, ' ', rec_id, ' ', first_name, ' ', last_name, ' ', sex,age, ' ', birth_place, ' ', house_num, ' ', county_desc, ' ', street_name, ' ', zip_code, ' ', phone_num) as val "
    "FROM 'data/NCVR_AF_clean.csv'"
).execute("select * from src1").fetchall()

[('0_22_9865350',
  '22 9865350 whitney baker female29 in 400 orange poplar 27510  '),
 ('0_40_12768214',
  '40 12768214 abbington pope female23 nc 1221 wake westview 27605  '),
 ('0_122_9112102',
  '122 9112102 rebecca wilkins female49 nc 811 new hanover magnolia 28428 8120512'),
 ('0_140_9704280',
  '140 9704280 justin brown male34 ca 3225 orange us hwy 70 27243  '),
 ('0_222_3198122',
  '222 3198122 stephanie eissens female38 nc 100 durham village circle 27713  '),
 ('0_240_94472',
  '240 94472 danielle peschon female28 tn 1049 alamance kelso 27215 6758354'),
 ('0_251_6640272',
  '251 6640272 michelle hinnant female45 nc 130 johnston braswell 27577 2024737'),
 ('0_340_10179287',
  '340 10179287 mark caccio male64 ct 213 pender scottsdale 28411 6008095'),
 ('0_351_6376559',
  '351 6376559 nancy hoover female55 ny 618 iredell isle of pines 28117  '),
 ('0_451_1422321',
  '451 1422321 thomas johnson male29 nc 501 caswell cherry grove 27379 2696260'),
 ('0_522_12340214',
  '522 12340214

In [68]:
con.execute("drop table if exists src2").execute(
    "CREATE TABLE src2 AS "
    "SELECT id as rid, concat(entity, ' ', rec_id, ' ', first_name, ' ', last_name, ' ', sex,age, ' ', birth_place, ' ', house_num, ' ', county_desc, ' ', street_name, ' ', zip_code, ' ', phone_num) as val "
    "FROM 'data/NCVR_BF_clean.csv'"
).execute("select * from src2").fetchall()

[('1_40_13913995',
  '40 13913995 abbington pope female23 nc 104 wayne breezewood 27534  '),
 ('1_41_520429',
  '41 520429 angelon smith female40 nc 119 buncombe flint 28801  '),
 ('1_122_9000404',
  '122 9000404 rebecca wilkins female49   1501 nash lafayette 27803  '),
 ('1_140_9350108',
  '140 9350108 justin brown male34 ca 211 new hanover queen 28401 7417817'),
 ('1_222_13226442',
  '222 13226442 stephanie eissens meacomes female38 nc 4033 wake enfield ridge 27519  '),
 ('1_240_116265',
  '240 116265 danielle peschon female28   730 alamance boone station 27215 2789980'),
 ('1_241_8517073',
  '241 8517073 lovie matthews female26 tn 5634 mecklenburg via romano 28270  '),
 ('1_322_6614729',
  '322 6614729 sandra creech female52 nc 1347 johnston crocker 27577 4645707'),
 ('1_340_9418808',
  '340 9418808 mark caccio male64 ct 605 onslow windsong north 28584 6008095'),
 ('1_341_6619987',
  '341 6619987 amanda spencer female37 nc 607 johnston preston 27576  '),
 ('1_422_10444451',
  '422 1

In [80]:
con.execute("drop table if exists src3").execute(
    "CREATE TABLE src3 AS "
    "SELECT id as rid, concat(entity, ' ', rec_id, ' ', first_name, ' ', last_name, ' ', sex,age, ' ', birth_place, ' ', house_num, ' ', county_desc, ' ', street_name, ' ', zip_code, ' ', phone_num) as val "
    "FROM 'data/NCVR_CF_clean.csv'"
).execute("select * from src3").fetchall()

[('2_22_3326652',
  '22 3326652 whitney baker female29 in 709 durham green 27701  '),
 ('2_31_3904957',
  '31 3904957 latonya mciver female40 va 30 forsyth glenwood 27106 9166714'),
 ('2_131_12315539',
  '131 12315539 alice feldbusch female29 nc 119 wake fawn 27587  '),
 ('2_140_12924542',
  '140 12924542 justin brown male34 ca 8510 wake silhouette 27613  '),
 ('2_222_12748029',
  '222 12748029 stephanie eissens female38 nc 133 wake wards ridge 27513  '),
 ('2_231_9728052',
  '231 9728052 william mcinerney male29 nc 265 orange severin 27516  '),
 ('2_240_13265262',
  '240 13265262 danielle peschon female28 tn 2319 wake hinton 27612  '),
 ('2_331_13927418',
  '331 13927418 james byrd male53 nc 610 wayne park 27530 9152245'),
 ('2_431_2134645',
  '431 2134645 ainslie guion female57 ky 323 craven trenton 28523 8144405'),
 ('2_440_12811452',
  '440 12811452 heather spence female33 nc 6009 wake splitrock 27539  '),
 ('2_522_12715739',
  '522 12715739 judy parker female56 nc 249 wake money t

In [87]:
con.execute("drop view if exists srcall").execute(
    "create view srcall as "
    "select * from src1 "
    "union "
    "select * from src2 "
    "union "
    "select * from src3 "
).execute("select * from srcall").fetchall()

[('2_22_3326652',
  '22 3326652 whitney baker female29 in 709 durham green 27701  '),
 ('2_31_3904957',
  '31 3904957 latonya mciver female40 va 30 forsyth glenwood 27106 9166714'),
 ('2_131_12315539',
  '131 12315539 alice feldbusch female29 nc 119 wake fawn 27587  '),
 ('2_140_12924542',
  '140 12924542 justin brown male34 ca 8510 wake silhouette 27613  '),
 ('2_222_12748029',
  '222 12748029 stephanie eissens female38 nc 133 wake wards ridge 27513  '),
 ('2_231_9728052',
  '231 9728052 william mcinerney male29 nc 265 orange severin 27516  '),
 ('2_240_13265262',
  '240 13265262 danielle peschon female28 tn 2319 wake hinton 27612  '),
 ('2_331_13927418',
  '331 13927418 james byrd male53 nc 610 wayne park 27530 9152245'),
 ('2_431_2134645',
  '431 2134645 ainslie guion female57 ky 323 craven trenton 28523 8144405'),
 ('2_440_12811452',
  '440 12811452 heather spence female33 nc 6009 wake splitrock 27539  '),
 ('2_522_12715739',
  '522 12715739 judy parker female56 nc 249 wake money t

In [88]:
# function args
l_table = 'srcall'
r_table = 'srcall'

l_key_attr = 'rid'
r_key_attr = 'rid'
l_join_attr = 'val'
r_join_attr = 'val'
# tokenizer = tokenizers.WordsTokzr()
tokenizer = tokenizers.QGramsTokzr(3)
threshold = 0.5
out_table_name = 'matches'
bf_out_table_name = 'bfmatches'

In [89]:
start_time = time.time()
jaccard_join_brute_force(
    con, l_table, r_table, l_key_attr, r_key_attr, l_join_attr, r_join_attr, tokenizer, threshold, bf_out_table_name
)
end_time = time.time()
end_time - start_time

0.002909421920776367

In [90]:
con.execute(f"select * from {bf_out_table_name}").fetchall()

[('0_4840_7246686', '2_4840_7261567'),
 ('1_4422_1365826', '2_4422_1365827'),
 ('0_3822_11913154', '1_3822_11891631'),
 ('0_1022_7968679', '2_1022_7999709'),
 ('0_1022_7968679', '1_1022_8529932'),
 ('1_922_12317422', '2_922_12843169'),
 ('1_1040_10950068', '2_1040_10986386'),
 ('0_1040_10986345', '2_1040_10986386'),
 ('1_1022_8529932', '2_1022_7999709'),
 ('0_922_12748976', '2_922_12843169'),
 ('0_1040_10986345', '1_1040_10950068'),
 ('0_1222_1130018', '1_1222_1104748'),
 ('0_4940_6724474', '2_4940_6769191'),
 ('1_4022_10236169', '2_4022_10221011'),
 ('0_922_12748976', '1_922_12317422'),
 ('0_1940_8280393', '2_1940_7342755')]

In [91]:
start_time = time.time()
jaccard_join(
    con, l_table, r_table, l_key_attr, r_key_attr, l_join_attr, r_join_attr, tokenizer, threshold, out_table_name
)
end_time = time.time()
end_time - start_time

0.01742410659790039

In [92]:
con.execute(f"select * from {out_table_name}").fetchall()

[('0_1022_7968679', '2_1022_7999709'),
 ('0_1040_10986345', '1_1040_10950068'),
 ('0_1040_10986345', '2_1040_10986386'),
 ('0_1222_1130018', '1_1222_1104748'),
 ('0_1940_8280393', '2_1940_7342755'),
 ('0_1022_7968679', '1_1022_8529932'),
 ('0_3822_11913154', '1_3822_11891631'),
 ('0_4940_6724474', '2_4940_6769191'),
 ('0_922_12748976', '2_922_12843169'),
 ('1_1022_8529932', '2_1022_7999709'),
 ('1_1040_10950068', '2_1040_10986386'),
 ('0_4840_7246686', '2_4840_7261567'),
 ('0_922_12748976', '1_922_12317422'),
 ('1_4422_1365826', '2_4422_1365827'),
 ('1_4022_10236169', '2_4022_10221011'),
 ('1_922_12317422', '2_922_12843169')]

In [93]:
con.execute(
    "select * "
    f"from {out_table_name} m "
    f"full outer join {bf_out_table_name} b on b.rid1 = m.rid1 and b.rid2 = m.rid2 "
    "where m.rid1 is null "
).fetchall()

[]